Let's build GPT from Scratch, in code, spelled out - Andrej Karpathy
https://www.youtube.com/watch?v=kCc8FmEb1nY&t=4057s

Transformer based, character level language model (nanoGPT)

#TODO: move computation to GPU

In [12]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [13]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [14]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [15]:
# create a mapping from characters to integers
#character level encoding - usually subword or byte-pair in practice 
# trade off between sequence length and vocab size
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [22]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [23]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

#we train on chunks of the training data, not the whole training data set
# block size is the max length of training sequence chunks
# a chunk of 9 examples has 8 training examples for the trainsformer to learn from (cumulatively concatenating sequence to predict next token)
# this enables the transformer to learn to predict from varying block sizes
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [40]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

# for t in range(block_size):
context = x[:1+1]
target = y[3]
print(f"when input is {context} the target: {target}")

when input is tensor([18, 47]) the target: 58


In [53]:
#now we configure batch info for feeding into the transformer

torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    #random index * batch size
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #stack batches for context
    x = torch.stack([data[i:i+block_size] for i in ix])
    #stack batches for prediction/loss
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # print(x,y)
    return x, y

xb, yb = get_batch('train')
#fix scalar tensor issue with variable indices
xb1 = xb.numpy()
yb1 = yb.numpy()
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb1[b, :t+1]
        target = yb1[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [ ]:
print(xb) # our input to the transformer

In [54]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

#we use a bigram language model for prediction

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        # V * V table of probability of next token given current token
        # forward pass will pluck out the row relevant to the current character
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # called at init when creating model instance
        # idx and targets are both (B,T) tensor of integers
        # pluck out logits for next forward pass
        logits = self.token_embedding_table(idx) # (Batch,Time,Channel) (4,8,V)
        
        if targets is None:
            loss = None
        else:
            #reshape dimensions for compatibility with CrossEntropy
            B, T, C = logits.shape
            #view predictions tensor -> make it 2D
            logits = logits.view(B*T, C)
            #view targets tensor -> make it 2D
            targets = targets.view(B*T)
            #negative log likelihood loss - how correct are our logits compared to the ground truth?
            #correct prediction logit should be high and the rest low
            #loss should begin at -log(1/V)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    #idx is current context of a batch
    #start with B*T and extend to B*T+1 for next token in time dimension for each batch
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions form forward
            logits, loss = self(idx)
            # focus only on the last time step
            # pluck out the last column of the logits 1 * B * V
            #currently bigram model, no point passing whole context window but in future will be needed
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities over Vocab for last token of logits
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution -> select single prediction for next token for each batch
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence of indices
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

#idx is the first char / initialization token newline
#max new tokens is 100
#zeroth row is first batch
#print out the sequence of 100 tokens produced from the initial starting token 0 / newline
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [56]:
# create a PyTorch optimizer
# pass in logits params and specify learning rate
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [62]:
#training simple bigram model - no context used for learning yet
batch_size = 32
for steps in range(100000): # increase number of steps for good results... 
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    #zero out gradients from previous loss
    optimizer.zero_grad(set_to_none=True)
    #get gradients for all parameters
    loss.backward()
    #use gradients to update params
    optimizer.step()

print(loss.item())




2.5146496295928955

Your hanfilllar athel--t trmingous minge Yo tor derdo by.
I ah quthrondondonckn gh keme;
Grachea gav


In [68]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))



Ay siee at, lapen agoowhare nord f iooomy wishe shnene

HE:
A y uce issor hin IDed
Smig the prinr sthit avinee,--reth,
Thastlitishe andulit d gst ad



Code w avifoitinde I

Ag Jo th: tho avar,
GHe
YBe mathathiebaye s mesethorcl crckicas terowire?
hel ptint ad NENER: tourunlin forer bloy.
is s my ino dinee myooorineat mptll.


hifesay, ccepr k, Whe herme tco thule athur so, id Ise'l

Wat I hanthayowss t hin, rou alendo y mo INThe
O:
BRICOLAs, mucu Bu y.
KENUS:
SBETofale bereiden the r sindis cot


The mathematical trick in self-attention

In [69]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [87]:
# We want x[b,t] = mean_{i<=t} x[b,i]
#we create a bag of words
xbow = torch.zeros((B,T,C)).numpy()
x1 = x.numpy()
#for each batch
for b in range(B):
    #for each timestep in batch
    for t in range(T):
        #prev timestep range up to and including t
        xprev = x1[b,:t+1] # (t,C)
        xprev = torch.from_numpy(xprev)
        #bag of words = average over previous timestep. This gives us a BoW over the vocab
        xbow[b,t] = torch.mean(xprev, 0)
xbow = torch.from_numpy(xbow)
x1[0]

array([[ 0.18077157, -0.06998809],
       [-0.3596235 , -0.91520447],
       [ 0.6257653 ,  0.02550992],
       [ 0.95451367,  0.06434853],
       [ 0.36115056,  1.1678782 ],
       [-1.3498932 , -0.5101767 ],
       [ 0.2359577 , -0.23977837],
       [-0.9211147 ,  1.543297  ]], dtype=float32)

In [88]:
# we see that xbow is producing averages over previous predictions
# each successive row averages over prev+curr
xbow[0]
#works but very inefficient, lets matmul

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [89]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
#tril is a triangular matrix - we use this for masking
a = torch.tril(torch.ones(3, 3))
#normalize values to sum to 1 - this gives us our attention weights
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
#c is the first row of a * first col of b
#3*3 x 3*2 (middle has to match, outside is what is created)
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [90]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [ ]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
#mask elements where tril is zero
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1) #softmax over each row for norm
xbow3 = wei @ x
torch.allclose(xbow, xbow3)
